<a href="https://colab.research.google.com/github/ignacioaranguren1/bd_2/blob/main/bd_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os
import keras_tuner

from tqdm import tqdm
from datetime import datetime
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from keras_tuner.tuners import RandomSearch

In [4]:
%matplotlib inline

datapath = '/Users/ignacioaranguren/bd_2/data'
os.chdir(datapath)

1.In the data used by Gu, Kelly and Xiu (RFS 2019 – provided in class), use a similar procedure to theirs to predict stock returns with neural networks. Start by finding a suitable baseline configuration, and use a validation procedure to pick optimal hyperparameters for three neural network models: One with 2 hidden layers, one with 3 hidden layers, and one with 4 hidden layers. 

In [5]:
data = pd.read_pickle('returns_chars_panel.pkl')
macro = pd.read_pickle('macro_timeseries.pkl')

In [6]:
def train_validation_test_split(data,train_end_date,validation_end_date):
  tmp = data.reset_index()
  train = tmp[tmp.date<=train_end_date].set_index(['date','permno'],drop=True)
  validation = tmp[(tmp.date>train_end_date) & (tmp.date<=validation_end_date)].set_index(['date','permno'],drop=True)
  test = tmp[tmp.date>validation_end_date].set_index(['date','permno'],drop=True)
  return train,validation,test

In [10]:
data_merged = pd.merge(data,macro,on=['date'])
datelist = list(set(data_merged['date']))
datelist.sort()
data_merged.set_index(['date','permno'],drop=True,inplace=True)

In [45]:
train_ratio = 0.5
validation_ratio = 0.25
train_date = datelist[int(len(datelist)*train_ratio)]
validation_date = datelist[int(len(datelist)*(train_ratio+validation_ratio))]
X = data_merged.iloc[:,3:].copy()
y = data_merged['excess_ret'].copy()

In [46]:
X_train,X_validation,X_test = train_validation_test_split(X,train_date,validation_date)
y_train,y_validation,y_test = train_validation_test_split(y,train_date,validation_date)

In [121]:
def fit(model, x, y, validation_data, **kwargs):
  model.fit(x, y, **kwargs)
  x_val, y_val = validation_data
  y_pred = model.predict(x_val)
  # Return a single float to minimize.
  return np.mean((y_pred - y_val)**2)

EPOCHS = 16
BATCH_SIZE = 256

def keras_code(n_layers, units, dropout_rate, learning_rate):
    # Build model
    model = Sequential()
    model.add(layers.Input(shape=(105,)))
    for i in range(n_layers):
        model.add(layers.Dense(units=units[i], activation='relu'))
    model.add(layers.Dropout(rate=dropout_rate))
    model.add(layers.Dense(1, activation='linear'))
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss='mse')
    return fit(
        model, 
        X_train.values, 
        y_train.values,
        validation_data=(X_validation.values, y_validation.values),
        batch_size=BATCH_SIZE,
        epochs=EPOCHS
        )
    
    
class MyTuner(keras_tuner.RandomSearch): 
    def __init__(self, n_layers, *args, **kwargs):
        self.n_layers = n_layers
        super().__init__(*args, **kwargs)
        
    def run_trial(self, trial, **kwargs):
        hp = trial.hyperparameters
        return keras_code(
            n_layers=self.n_layers,
            units=[hp.Int(f'units_{i + 1}',min_value=16,max_value=160,step=16) for i in range(self.n_layers)],
            dropout_rate=hp.Choice('dropout', values=[0.2,0.4,0.6,0.8]),
            learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
        )
    
    

In [126]:
MAX_TRIALS = 10
EXECUTION_PER_TRIAL = 2

def tune_model(n_layers):
  tuner = MyTuner(
      n_layers,
      max_trials=MAX_TRIALS,
      executions_per_trial=EXECUTION_PER_TRIAL,
      overwrite=True,
      directory='my_dir',
      project_name=f'NN_new_{n}'
  )
  tuner.search()
  return tuner

In [ ]:
tuner = tune_model(3)

Trial 2 Complete [00h 02m 07s]
default_objective: 0.04026697128396721

Best default_objective So Far: 0.04026697128396721
Total elapsed time: 00h 03m 44s

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
112               |128               |units_1
128               |96                |units_2
96                |16                |units_3
0.6               |0.8               |dropout
0.0035373         |0.007101          |lr

Epoch 1/16
4920/4920 [==============================] - 7s 1ms/step - loss: 0.0227
Epoch 2/16
4920/4920 [==============================] - 7s 2ms/step - loss: 0.0225
Epoch 3/16
4920/4920 [==============================] - 7s 1ms/step - loss: 0.0225
Epoch 4/16
4920/4920 [==============================] - 7s 1ms/step - loss: 0.0224
Epoch 5/16
4920/4920 [==============================] - 7s 1ms/step - loss: 0.0224
Epoch 6/16
4920/4920 [==============================] - 7s 1ms/step - loss: 0.0223
Epoch 7/16
4920/4920 [====================

In [120]:
tuner.get_best_hyperparameters()[0].values

{'units1': 16, 'units2': 16, 'units3': 16, 'dropout': 0.2, 'lr': 0.0001}

In [32]:
models=[]
parameters=[]
tuners=[]
for n in range(2,5):
  tuner = tune_model(n)
  parameters.append(tuner.get_best_hyperparameters)
  models.append(tuner.get_best_models(1))
  tuners.append(tuner)


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
32                |?                 |units_1
80                |?                 |units_2
0.8               |?                 |dropout
0.0020594         |?                 |lr

Epoch 1/16


ValueError: in user code:

    File "/usr/local/lib/python3.9/site-packages/keras/engine/training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.9/site-packages/keras/engine/training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.9/site-packages/keras/engine/training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/lib/python3.9/site-packages/keras/engine/training.py", line 889, in train_step
        y_pred = self(x, training=True)
    File "/usr/local/lib/python3.9/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/usr/local/lib/python3.9/site-packages/keras/engine/input_spec.py", line 264, in assert_input_compatibility
        raise ValueError(f'Input {input_index} of layer "{layer_name}" is '


In [ ]:
with open('parameters.pkl','wb') as f:
  pickle.dump(parameters,f,protocol=4)
with open('models.pkl','wb') as f:
  pickle.dump(parameters,f,protocol=4)
with open('tuners.pkl','wb') as f:
  pickle.dump(parameters,f,protocol=4)

In [ ]:
model = Sequential()
model.add(layers.Dense(16, input_dim=2, activation='relu'))
model.add(layers.Dense(16, input_dim=2, activation='relu'))
model.add(layers.Dense(1, activation='linear'))
model.compile(loss='mse', optimizer='adam')
model.fit(X, y, epochs=16, batch_size=64,verbose=True)

In [ ]:
train_result = model.evaluate(X_train.values,y_train.values,batch_size=10000)
test_result = model.evaluate(X_test.values,y_test.values)
print('TRAIN MSE:',train_result)
print('TEST MSE:',test_result)

34491/34491 [==============================] - 80s 2ms/step - loss: 0.0259
TRAIN MSE: 0.022671449929475784
TEST MSE: 0.025928305462002754


In [ ]:
test_result

0.025928305462002754

2.Use test data to get an idea of the out of sample performance of each model. Convert the standard MSE metric for out of sample performance to the “R2 out of sample” metric that was discussed in class. Compare your results to those in Gu-Kelly-Xiu and comment on the differences. 

In [ ]:
# from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
# import eli5
# from eli5.sklearn import PermutationImportance

# def base_model():
#     model = Sequential()        
#     ...
#     return model

# X = ...
# y = ...

# my_model = KerasRegressor(build_fn=base_model, **sk_params)    
# my_model.fit(X,y)

# perm = PermutationImportance(my_model, random_state=1).fit(X,y)
# eli5.show_weights(perm, feature_names = X.columns.tolist())

3.Pick the model that performs the best out of sample, and interpret its output by doing the following analysis of variable importance:
a.	First, for all stock characteristics, get variable importance by setting one predictor at a time to zero and finding the decrease in out of sample R2. Show a table of the 10 most important variables according to this measure, and give an economic interpretation. 


b.	Second, get a measure of the joint importance of all our “macro predictors” (i.e., those taken from Welch and Goyal 2008), by setting them all to zero and finding the decrease in out of sample R2. Comment on how important macroeconomic variables are relative to stock characteristics in predicting returns. 

c.	Repeat the two steps above, but by using a measure of the sensitivity of predictions to each input variable, as outlined in the lectures.

4.Fit a penalised linear model (LASSO) to the same data, using validation data to pick the best penalty (e.g., you can use the “sklearn” package in Python to do this easily). Compare its test data performance to the neural network. 

5.Suppose somebody tells you to collect 10 more micro or macro variables that can predict returns and are not in our current dataset. How would you choose those variables, based on the intuitions you have gained in this project?